## pytorch data loader

In [ ]:
from visar.pytorch_utils import compound_FP_loader

In [ ]:
para_dict = {
    'model_name': 'pyDNN_reg',
    'task_list': ['T107','T108'],
    'eval_type': 'regression',
    # input data related params:
    'dataset_file': './data/MT_data_clean_June28.csv',
    'feature_type': 'Morgan',
    'id_field': 'molregno',
    'smiles_field': 'salt_removed_smi',
    'model_flag': 'MT',
    'add_features': None,
    'frac_train': 0.8,
    'rand_seed': 0,
    'batch_size': 100,
    # model architecture related parameters:
}

In [ ]:
train_loader, test_loader, train_df, test_df = compound_FP_loader(para_dict)

In [ ]:
X, y, w, ids, smiles = next(iter(test_loader))

## pytorch data loader Attentive FP

In [1]:
from visar.AttentiveFP_utils import feature_dict_loader
para_dict = {
    'model_name': 'Attentive_FP_reg',
    'task_list':['T11409', 'T10612'],
    # input data related params:
    'dataset_file': './data/kinase_sample_data_new_processed.csv',
    'feature_file': './data/kinase_sample_data_new.pickle.pickle',
    'smiles_field': 'cano_smiles',
    'id_field': 'molregno',
    'num_atom_features': 39,
    'num_bond_features': 10,
    'model_flag': 'ST',
    'add_features': None,
    'frac_train': 0.8,
    'rand_seed': 0,
    'batch_size': 100,
    # model architecture related parameters:
    'radius': 2,
    'T': 1,
    'fingerprint_dim': 128,
    'output_units_num': 2,
    'dropouts': 0.4,
    'batch_normalization': True,    
}

In [2]:
train_loader, test_loader, train_df, test_df = feature_dict_loader(para_dict)

Extracted dataset shape: (1496, 4)


## pytorch model training

In [6]:
384 / 32.

12.0